# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
M. Schirmer  ->  M. Schirmer  |  ['M. Schirmer']
I. J. M. Crossfield  ->  I. J. M. Crossfield  |  ['I. J. M. Crossfield']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
R. Burn  ->  R. Burn  |  ['R. Burn']
J. Liu  ->  J. Liu  |  ['J. Liu']


X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Li  ->  J. Li  |  ['J. Li']
M. Feldt  ->  M. Feldt  |  ['M. Feldt']
T. Bertram  ->  T. Bertram  |  ['T. Bertram']
H. Coppejans  ->  H. Coppejans  |  ['H. Coppejans']
M. Kulas  ->  M. Kulas  |  ['M. Kulas']
S. Scheithauer  ->  S. Scheithauer  |  ['S. Scheithauer']
H. Steuer  ->  H. Steuer  |  ['H. Steuer']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
J. Shi  ->  J. Shi  |  ['J. Shi']
H. Klahr  ->  H. Klahr  |  ['H. Klahr']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 77 new papers today
          11 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/11 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2411.16808


extracting tarball to tmp_2411.16808... done.


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:488: LatexWarning: Error parsing the document directly. Trying to recover.
  warnings.warn(LatexWarning(f"Error parsing the document directly. Trying to recover."))


✔ → 0:header
  ↳ 32269:\section{\label{sc:Intro}Introduction}
✔ → 32269:\section{\label{sc:Intro}Introduction}
  ↳ 38751:\section{Early Release Observations}\label{sec: Datasets}


...

Retrieving document from  https://arxiv.org/e-print/2411.17408
extracting tarball to tmp_2411.17408...

 done.


M. Benisty  ->  M. Benisty  |  ['M. Benisty']


Found 81 bibliographic references in tmp_2411.17408/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2411.17514
extracting tarball to tmp_2411.17514... done.
Retrieving document from  https://arxiv.org/e-print/2411.17516


extracting tarball to tmp_2411.17516... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.16808-b31b1b.svg)](https://arxiv.org/abs/2411.16808) | **Euclid: Searches for strong gravitational lenses using convolutional neural nets in Early Release Observations of the Perseus field**  |
|| R. Pearce-Casey, et al. -- incl., <mark>K. Jahnke</mark>, <mark>M. Schirmer</mark> |
|*Appeared on*| *2024-11-27*|
|*Comments*| *22 pages, 11 figures, Euclid consortium paper, A&A submitted*|
|**Abstract**|            The Euclid Wide Survey (EWS) is predicted to find approximately 170 000 galaxy-galaxy strong lenses from its lifetime observation of 14 000 deg^2 of the sky. Detecting this many lenses by visual inspection with professional astronomers and citizen scientists alone is infeasible. Machine learning algorithms, particularly convolutional neural networks (CNNs), have been used as an automated method of detecting strong lenses, and have proven fruitful in finding galaxy-galaxy strong lens candidates. We identify the major challenge to be the automatic detection of galaxy-galaxy strong lenses while simultaneously maintaining a low false positive rate. One aim of this research is to have a quantified starting point on the achieved purity and completeness with our current version of CNN-based detection pipelines for the VIS images of EWS. We select all sources with VIS IE < 23 mag from the Euclid Early Release Observation imaging of the Perseus field. We apply a range of CNN architectures to detect strong lenses in these cutouts. All our networks perform extremely well on simulated data sets and their respective validation sets. However, when applied to real Euclid imaging, the highest lens purity is just 11%. Among all our networks, the false positives are typically identifiable by human volunteers as, for example, spiral galaxies, multiple sources, and artefacts, implying that improvements are still possible, perhaps via a second, more interpretable lens selection filtering stage. There is currently no alternative to human classification of CNN-selected lens candidates. Given the expected 10^5 lensing systems in Euclid, this implies 10^6 objects for human classification, which while very large is not in principle intractable and not without precedent.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.16836-b31b1b.svg)](https://arxiv.org/abs/2411.16836) | **OrCAS: Origins, Compositions, and Atmospheres of Sub-neptunes. I. Survey Definition**  |
|| <mark>I. J. M. Crossfield</mark>, et al. -- incl., <mark>L. Kreidberg</mark> |
|*Appeared on*| *2024-11-27*|
|*Comments*| *20 pages, 4 figures, 4 tables, 26 sub-Neptunes, 31 TOIs. Accepted to AJ*|
|**Abstract**|            Sub-Neptunes - volatile-rich exoplanets smaller than Neptune - are intrinsically the most common type of planet known. However, the formation and nature of these objects, as well as the distinctions between sub-classes (if any), remain unclear. Two powerful tools to tease out the secrets of these worlds are measurements of (i) atmospheric composition and structure revealed by transit and/or eclipse spectroscopy, and (ii) mass, radius, and density revealed by transit photometry and Doppler spectroscopy. Here we present OrCAS, a survey to better elucidate the origins, compositions, and atmospheres of sub-Neptunes. This radial velocity survey uses a repeatable, quantifiable metric to select targets suitable for subsequent transmission spectroscopy and address key science themes about the atmospheric & internal compositions and architectures of these systems. Our survey targets 26 systems with transiting sub-Neptune planet candidates, with the overarching goal of increasing the sample of such planets suitable for subsequent atmospheric characterization. This paper lays out our survey's science goals, defines our target prioritization metric, and performs light-curve fits and statistical validation using existing TESS photometry and ground-based follow-up observations. Our survey serves to continue expanding the sample of small exoplanets with well-measured properties orbiting nearby bright stars, ensuring fruitful studies of these systems for many years to come.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.16879-b31b1b.svg)](https://arxiv.org/abs/2411.16879) | **Water-rich sub-Neptunes and rocky super Earths around different Stars: Radii shaped by Volatile Partitioning, Formation, and Evolution**  |
|| <mark>R. Burn</mark>, et al. |
|*Appeared on*| *2024-11-27*|
|*Comments*| *25 pages, 17 figures, submitted to A&A*|
|**Abstract**|            The nature of sub-Neptunes remains unknown due to degeneracies in interior structure solutions. However, a statistical set of small planets with measured masses and radii has been compiled. It can be used to test the prediction of large water reservoirs on sub-Neptunes by planet formation theory. We want to find out whether this water reservoir is included in photoevaporative winds and how much of it can partition into the rocky and metallic interior. We couple the result of a planetary formation model to evolution models which assume perfect mixing of water with H/He in the envelope or complete segregation. For the mixed envelopes, we also include fractionation during photoevaporative mass-loss. Further, the effect of equilibrium dissolution of water into an assumed magma ocean and into the metallic core is studied for the first time in coupled formation-evolution models. Out of the tested scenarios, the mass-radius relation of exoplanets is best matched under the mixed assumption without water sequestration to the interior. We quantify the radius valley location and scaling with stellar mass. Fractionation is not found to significantly alter the composition of the planets for our initial conditions due to initially massive envelopes on all planets. In contrast, water sequestration has a profound effect on the radius evolution and compositional budget of the planets. The model predicts the preservation of large quantities of water even if the gaseous envelope is lost. Planets with corresponding bulk densities are not observed in comparably large numbers. By combining formation and evolution model, we probe a parameter space favored by core accretion theory. We conclude that the dissolution of different volatiles into the planetary interior and solidification of the magma ocean are natural next steps for comprehensive treatment of atmosphere-interior interaction. (abridged)         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.17408-b31b1b.svg)](https://arxiv.org/abs/2411.17408) | **Gas dynamics around a Jupiter mass planet: II. Chemical evolution of circumplanetary material**  |
|| A. J. Cridland, E. Lega, <mark>M. Benisty</mark> |
|*Appeared on*| *2024-11-27*|
|*Comments*| *20 pages, 17 figures, accepted for publication in A&A*|
|**Abstract**|            In an ongoing effort to understand planet formation the link between the chemistry of the protoplanetary disk and the properties of resulting planets have long been a subject of interest. These connections have generally been made between mature planets and young protoplanetary disks through the carbon-to-oxygen (C/O) ratio. In a rare number of systems, young protoplanets have been found within their natal protoplanetary disks. These systems offer a unique opportunity to directly study the delivery of gas from the protoplanetary disk to the planet. In this work we post-process 3D numerical simulations of an embedded Jupiter-massed planet in its protoplanetary disk to explore the chemical evolution of gas as it flows from the disk to the planet. The relevant dust to this chemical evolution is assumed to be small, co-moving grains with a reduced dust-to-gas ratio indicative of the upper atmosphere of a protoplanetary disk. We find that as the gas enters deep into the planet's gravitational well, it warms significantly (up to $\sim 800$ K), releasing all of the volatile content from the ice phase. This change in phase can influence our understanding of the delivery of volatile species to the atmospheres of giant planets. The primary carbon, oxygen, and sulfur carrying ices: CO$_2$, H$_2$O, and H$_2$S are released into the gas phase and along with the warm gas temperatures near the embedded planets lead to the production of unique species like CS, SO, and SO$_2$ compared to the protoplanetary disk. We compute the column densities of SO, SO$_2$, CS, and H$_2$CS in our model and find that their values are consistent with previous observational studies.         |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.17008-b31b1b.svg)](https://arxiv.org/abs/2411.17008) | **SN 2014C: a metamorphic supernova exploded in the intricate and hydrogen-rich surroundings**  |
|| Q. Zhai, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2024-11-27*|
|*Comments*| *23 pages, 15 figures, submitted to ApJ*|
|**Abstract**|            We present photometric and spectroscopic observations of supernova (SN) 2014C, primarily emphasizing the initial month after the explosion at approximately daily intervals. During this time, it was classified as a Type Ib SN exhibiting a notably higher peak luminosity ($L_{\rm peak}\approx4.3\times10^{42}\rm erg\,s^{-1}$), a faster rise to brightness ($t_{\rm rise} \approx 11.6$ d), and a more gradual dimming ($\Delta m_{15}^{V} \approx 0.48$ mag) compared to typical SNe Ib. Analysis of the velocity evolution over the first $\sim$ 20 days after the explosion supports the view that the absorption near 6200Å\,is due to high-velocity H$\alpha$ in the outer layers of the ejecta, indicating the presence of a small amount of hydrogen in the envelope of progenitor before the explosion. Assuming the peak luminosity is entirely attributed to radioactive decay, we estimate that 0.14 \Msun\,of \Nifs\,was synthesized in the explosion. However, this amount of nickel could no longer maintain observed brightness approximately ten days after peak luminosity, suggesting additional energy sources beyond radioactive decay. This supplementary energy likely originates from interaction with the circumstellar medium (CSM). Consequently, the timing of the SN-CSM interaction in SN 2014C may occur much earlier than the emergence of IIn-like features during the nebular phase.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.17094-b31b1b.svg)](https://arxiv.org/abs/2411.17094) | **Analytical modeling of the one-dimensional power spectrum of 21-cm forest based on a halo model method**  |
|| Y. Shao, T.-Y. Sun, M.-L. Zhao, <mark>X. Zhang</mark> |
|*Appeared on*| *2024-11-27*|
|*Comments*| *10 pages, 7 figures*|
|**Abstract**|            The 21-cm forest, composed of spectral absorption features from high-redshift background radio sources, provides a unique probe for studying small-scale structures during the epoch of reionization. It is particularly sensitive to detecting small-scale structures and early heating processes. Despite the rich information contained in the 21-cm forest signal, the complexity of directly modeling the signal has led to a lack of effective analytical models. However, the one-dimensional (1D) power spectrum of the 21-cm forest contains valuable information about the matter power spectrum, making analytical modeling feasible. This work employs an analytical modeling approach based on the halo model, which links the distribution of matter to dark matter halos, allowing for effective predictions of cosmic structure formation and its impact on the 21-cm signal. By considering various parameter scenarios within the halo model framework, particularly different dark matter particle masses and varying levels of cosmic heating, we can capture the complexities of small-scale structures and make the 1D power spectrum modeling applicable across a wide range of parameters. This method not only enhances our understanding of the 21-cm forest signal but also provides theoretical support for future observational data. Observing the 21-cm forest with large radio telescopes, such as the Square Kilometre Array, is anticipated to enable simultaneous exploration of dark matter properties and the heating history of the early universe.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.17121-b31b1b.svg)](https://arxiv.org/abs/2411.17121) | **The deuterium fractionation of NH$_3$ in massive star-forming regions**  |
|| Y. Li, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-11-27*|
|*Comments*| *Accepted for publication in ApJ. 34 pages, 25 figures, 7 tables*|
|**Abstract**|            Deuteration is sensitive to environmental conditions in star-forming regions. To investigate NH$_2$D chemistry, we compared the spatial distribution of ortho-NH$_2$D $1_{11}^s-1_{01}^a$, NH$_3$(1,1) and NH$_3$(2,2) in 12 late-stage massive star-forming regions. By averaging several pixels along the spatial slices of ortho-NH$_2$D $1_{11}^s-1_{01}^a$, we obtained the deuterium fractionation of NH$_3$. In seven targets, the deuterium fractionation of NH$_3$ shows a decreasing trend with increasing rotational temperature. This trend is less clear in the remaining five sources, likely due to limited spatial resolution. However, when considering all 12 sources together, the anticorrelation between NH$_3$ deuterium fractionation and rotational temperature becomes less significant, suggesting that other physical parameters may influence the fractionation. Additionally, we found that the region of highest deuterium fractionation of NH$_3$ is offset from the NH$_3$ peak in each source, likely because the temperature is higher near the NH$_3$ peaks and NH$_2$D may be depleted from the gas phase as the molecular cloud core evolves, as well as the increased release of CO from grains into the gas phase.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.17516-b31b1b.svg)](https://arxiv.org/abs/2411.17516) | **The BINGO/ABDUS Project: Forecast for cosmological parameter from a mock Fast Radio Bursts survey**  |
|| <mark>X. Zhang</mark>, et al. |
|*Appeared on*| *2024-11-27*|
|*Comments*| *11 pages, 10 figures, 2 tables*|
|**Abstract**|            There are hosts of surveys that will provide excellent data to search for and locate Fast Radio Bursts (FRBs) at cosmological distances. The BINGO project is one such surveys, and this collaboration has already estimated a FRB detection rate that the project will yield with the main telescope helped by a set of outrigger stations. This paper aims to simulate and estimate the potential of FRBs in constraining our current cosmological model. We present a forecast of the future constraints that the BINGO FRB detections and localizations will have when added to other current cosmological datasets. We quantify the dispersion measure (DM) as a function of redshift ($z$) for the BINGO FRB mock sample. Furthermore, we use current datasets (Supernovae, Baryonic Acoustic Oscillations, and Cosmic Microwave Background data) prior to assessing the efficacy of constraining dark energy models using Monte Carlo methods. Our results show that spatially located BINGO FRB dataset will provide promising constraints on the population of host galaxies intrinsic DM and be able to measure the nuisance parameters present within a FRB cosmological analysis. Still, they will also provide alternative estimates on other parameters such as the Hubble constant and the dark energy equation of state. In particular, we should see that BINGO FRB data can put constraints on the degenerate $w-H_0$ plane, which the CMB is incapable of measuring, allowing FRBs to be a viable alternative to BAO to constrain the dark energy equation of state. We conclude that FRBs remain a promising future probe for cosmology and that the FRBs detected by the BINGO project will contribute significantly to our knowledge of the current cosmological model.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.17341-b31b1b.svg)](https://arxiv.org/abs/2411.17341) | **High strehl and high contrast for the ELT instrument METIS - Final design, implementation, and predicted performance of the single-conjugate adaptive optics system**  |
|| <mark>M. Feldt</mark>, et al. -- incl., <mark>T. Bertram</mark>, <mark>H. Coppejans</mark>, <mark>M. Kulas</mark>, <mark>S. Scheithauer</mark>, <mark>H. Steuer</mark> |
|*Appeared on*| *2024-11-27*|
|*Comments*| *51 pages, 29 figures, accepted to Experimental Astronomy*|
|**Abstract**|            The Mid-infrared ELT Imager and Spectrograph (METIS) is a first-generation instrument for the Extremely Large Telescope (ELT), Europe's next-generation 39 m ground-based telescope for optical and infrared wavelengths. METIS will offer diffraction-limited imaging, low- and medium-resolution slit spectroscopy, and coronagraphy for high-contrast imaging between 3 and 13 microns, as well as high-resolution integral field spectroscopy between 3 and 5 microns. The main METIS science goals are the detection and characterisation of exoplanets, the investigation of proto-planetary disks, and the formation of planets. The Single-Conjugate Adaptive Optics (SCAO) system corrects atmospheric distortions and is thus essential for diffraction-limited observations with METIS. Numerous challenging aspects of an ELT Adaptive Optics (AO) system are addressed in the mature designs for the SCAO control system and the SCAO hardware module: the complex interaction with the telescope entities that participate in the AO control, wavefront reconstruction with a fragmented and moving pupil, secondary control tasks to deal with differential image motion, non-common path aberrations and mis-registration. A K-band pyramid wavefront sensor and a GPU-based Real-Time Computer (RTC), tailored to the needs of METIS at the ELT, are core components. This current paper serves as a natural sequel to our previous work presented in Hippler et al. (2018). It includes updated performance estimations in terms of several key performance indicators, including achieved contrast curves. We outline all important design decisions that were taken, and present the major challenges we faced and the main analyses carried out to arrive at these decisions and eventually the final design. We also elaborate on our testing and verification strategy, and, last not least, comprehensively present the full design, hardware and software.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error bad escape \e at position 25</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2411.16808/./PDFs/ROC_V2.png', 'tmp_2411.16808/./PDFs/HistogramZoobot.png', 'tmp_2411.16808/./PDFs/GradeA.png']
copying  tmp_2411.16808/./PDFs/ROC_V2.png to _build/html/
copying  tmp_2411.16808/./PDFs/HistogramZoobot.png to _build/html/
copying  tmp_2411.16808/./PDFs/GradeA.png to _build/html/
exported in  _build/html/2411.16808.md
    + _build/html/tmp_2411.16808/./PDFs/ROC_V2.png
    + _build/html/tmp_2411.16808/./PDFs/HistogramZoobot.png
    + _build/html/tmp_2411.16808/./PDFs/GradeA.png
found figures ['tmp_2411.16836/./triptych_ksms_26targs.png', 'tmp_2411.16836/./triptych_ksms_26targs.png', 'tmp_2411.16836/./sample_distribution_fits_v2.png']
copying  tmp_2411.16836/./triptych_ksms_26targs.png to _build/html/
copying  tmp_2411.16836/./triptych_ksms_26targs.png to _build/html/
copying  tmp_2411.16836/./sample_distribution_fits_v2.png to _build/html/
exported in  _build/html/2411.16836.md
    + _build/html/tmp_2411.16836/./triptych_ksms_26targs.png
    + _build/ht

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]$</div>



<div id="title">

# $\Euclid$: Searches for strong gravitational lenses using convolutional neural nets in Early Release Observations   of the Perseus field   $\thanks{This paper is published on     behalf of the Euclid Consortium.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2411.16808-b31b1b.svg)](https://arxiv.org/abs/2411.16808)<mark>Appeared on: 2024-11-27</mark> -  _22 pages, 11 figures, Euclid consortium paper, A&A submitted_

</div>
<div id="authors">

R. Pearce-Casey, et al. -- incl., <mark>K. Jahnke</mark>, <mark>M. Schirmer</mark>

</div>
<div id="abstract">

**Abstract:** The Euclid Wide Survey (EWS) is predicted to find approximately $\num{170000}$ galaxy-galaxy strong lenses from its lifetime observation of $\num{14000}\textrm{ deg}^2$ of the sky. Detecting this many lenses by visual inspection with professional astronomers and citizen scientists alone is infeasible. As a result, machine learning algorithms, particularly convolutional neural networks (CNNs), have been used as an automated method of detecting strong lenses, and have proven fruitful in finding galaxy-galaxy strong lens candidates, such that the usage of CNNs in lens identification has increased. We identify the major challenge to be the automatic detection of galaxy-galaxy strong lenses while simultaneously maintaining a low false positive rate, thus producing a pure and complete sample of strong lens candidates from $\Euclid$ with a limited need for visual inspection. One aim of this research is to have a quantified starting point on the achieved purity and completeness with our current version of CNN-based detection pipelines for the VIS images of EWS. This work is vital in preparing our CNN-based detection pipelines to be able to produce a pure sample of the $>\num{100000}$ strong gravitational lensing systems widely predicted for $\Euclid$ . We select all sources with VIS $\IE<23$ mag from the $\Euclid$ Early Release Observation imaging of the Perseus field. We apply a range of CNN architectures to detect strong lenses in these cutouts. All our networks perform extremely well on simulated data sets and their respective validation sets. However, when applied to real $\Euclid$ imaging, the highest lens purity is just $\sim11$ \% . Among all our networks, the false positives are typically identifiable by human volunteers as, for example, spiral galaxies, multiple sources, and artifacts, implying that improvements are still possible, perhaps via a second, more interpretable lens selection filtering stage. There is currently no alternative to human classification of CNN-selected lens candidates. Given the expected $\sim10^5$ lensing systems in $\Euclid$ , this implies $10^6$ objects for human classification, which while very large is not in principle intractable and not without precedent.

</div>

<div id="div_fig1">

<img src="tmp_2411.16808/./PDFs/ROC_V2.png" alt="Fig10" width="100%"/>

**Figure 10. -** ROC curves for the networks described in Section \ref{sec: Methods}.  Performance of the \texttt{Zoobot} networks are shown in panel (a), \texttt{Naberrie} and \texttt{MRC} networks are shown in (b), \texttt{NG}, \texttt{4-layer CNN}, and \texttt{ResNet-18} are shown in (c), and \texttt{Denselens} and \texttt{LencCLR} are shown in (d). The performance of the networks described in Sect. \ref{ssec:Leuzzi} are shown in (e), and (f) shows the \texttt{CBEN} networks at two voting values $\beta=0.3$ and $\beta=0.0001$, respectively. The orange diagonal line marked represents the expected performance of an entirely random classifier. The values in the legend of each subplot represent the true positive recovery for each network. Higher performance is represented in general by curves approaching the top-left corner of the plots. The best performing gravitational lens classifiers are the \texttt{Zoobot1} and \texttt{LensCLR} networks. (*fig:ROCAUC*)

</div>
<div id="div_fig2">

<img src="tmp_2411.16808/./PDFs/HistogramZoobot.png" alt="Fig5" width="100%"/>

**Figure 5. -** Distribution of predicted scores by \texttt{Zoobot1}(left) and \texttt{Zoobot2}(right) for lens candidates across grades $A$, $B$, and $C$. Grade $A$ lens candidates are confidently identified by both networks, shown to the right of the distribution in red at a high value of $p_{\sfont{LENS}}$. Grade $B$ predicted scores are shown in blue and are more varied for \texttt{Zoobot2}. The predicted scores for grade $C$ lens candidates have the most variation amongst grades, shown in green, for both networks illustrating that \texttt{Zoobot1} and \texttt{Zoobot2} are not as confident in identifying grade $C$ lens candidates. (*fig:Zoobot Distribution*)

</div>
<div id="div_fig3">

<img src="tmp_2411.16808/./PDFs/GradeA.png" alt="Fig1" width="100%"/>

**Figure 1. -** The three grade $A$ candidate lenses found by a visual inspection of $\num{12086}$ stamps from the Perseus cluster. The grade $A$ candidates show clear lensing features. The \IE-band images are $10^{\prime\prime}\times10^{\prime\prime}$. Detailed lens models for these candidates are shown in Appendix C of \cite{barroso2024euclidearlyreleaseobservations}. (*fig:grade a*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2411.16808"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\psj}{PSJ}$
$\newcommand{\ntarg}{26\xspace}$
$\newcommand{\ntoi}{31\xspace}$
$\newcommand{\vsini}{\ensuremath{v \sin i}\xspace}$
$\newcommand{\teff}{\ensuremath{T_\mathrm{eff}}\xspace}$
$\newcommand{\kms}{km~s\ensuremath{^{-1}}\xspace}$
$\newcommand{\kzz}{\ensuremath{K_{zz}}}$
$\newcommand{\soo}{SO_2}$
$\newcommand{\coo}{CO_2}$
$\newcommand{\water}{H_2O}$
$\newcommand{\methane}{CH_4}$
$\newcommand{\co}[2]{\ensuremath{^{#1}\textrm{C}^{#2}\textrm{O}}\xspace}$
$\newcommand{\car}[1]{\ensuremath{^{#1}\textrm{C}}\xspace}$
$\newcommand{\ox}[1]{\ensuremath{^{#1}\textrm{O}}\xspace}$
$\newcommand{\update}$
$\newcommand{\tirr}{\ensuremath{T_\mathrm{irr}}}$
$\newcommand{\TESS}{{\em TESS}}$
$\newcommand{\tess}{{\em TESS}}$
$\newcommand{\spitzer}{{\em Spitzer}}$
$\newcommand{\gj}{GJ~1252}$
$\newcommand{\gjb}{GJ~1252b}$
$\newcommand{\rp}{1.180}$
$\newcommand{\urp}{\pm0.078}$
$\newcommand{\vesc}{5.4}$
$\newcommand{\uvesc}{\pm0.8}$
$\newcommand{\tday}{1410}$
$\newcommand{\utday}{^{+91}_{-125}}$
$\newcommand{\depth}{149}$
$\newcommand{\udepth}{^{+25}_{-32}}$
$\newcommand{\te}{2458668.3575}$
$\newcommand{\ute}{^{+0.0019}_{-0.0007}}$
$\newcommand{\dt}{+1.4}$
$\newcommand{\udt}{^{+2.8}_{-1.0}}$
$\newcommand{\abUL}{0.41}$
$\newcommand{\fLL}{0.40}$
$\newcommand{\fabLL}{0.37}$
$\newcommand{\ecoswnosign}{0.0025}$
$\newcommand{\ecosw}{+0.0025}$
$\newcommand{\uecosw}{^{+0.0049}_{-0.0018}}$
$\newcommand{\psurfUL}{8.9}$
$\newcommand{\age}{3.9}$
$\newcommand{\uage}{0.4}$
$\newcommand{\referee}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\psj}{PSJ}$
$\newcommand{\ntarg}{26\xspace}$
$\newcommand{\ntoi}{31\xspace}$
$\newcommand{\vsini}{\ensuremath{v \sin i}\xspace}$
$\newcommand{\teff}{\ensuremath{T_\mathrm{eff}}\xspace}$
$\newcommand{\kms}{km~s\ensuremath{^{-1}}\xspace}$
$\newcommand{\kzz}{\ensuremath{K_{zz}}}$
$\newcommand{\soo}{SO_2}$
$\newcommand{\coo}{CO_2}$
$\newcommand{\water}{H_2O}$
$\newcommand{\methane}{CH_4}$
$\newcommand{\co}[2]{\ensuremath{^{#1}\textrm{C}^{#2}\textrm{O}}\xspace}$
$\newcommand{\car}[1]{\ensuremath{^{#1}\textrm{C}}\xspace}$
$\newcommand{\ox}[1]{\ensuremath{^{#1}\textrm{O}}\xspace}$
$\newcommand{\update}$
$\newcommand{\tirr}{\ensuremath{T_\mathrm{irr}}}$
$\newcommand{\TESS}{{\em TESS}}$
$\newcommand{\tess}{{\em TESS}}$
$\newcommand{\spitzer}{{\em Spitzer}}$
$\newcommand{\gj}{GJ~1252}$
$\newcommand{\gjb}{GJ~1252b}$
$\newcommand{\rp}{1.180}$
$\newcommand{\urp}{\pm0.078}$
$\newcommand{\vesc}{5.4}$
$\newcommand{\uvesc}{\pm0.8}$
$\newcommand{\tday}{1410}$
$\newcommand{\utday}{^{+91}_{-125}}$
$\newcommand{\depth}{149}$
$\newcommand{\udepth}{^{+25}_{-32}}$
$\newcommand{\te}{2458668.3575}$
$\newcommand{\ute}{^{+0.0019}_{-0.0007}}$
$\newcommand{\dt}{+1.4}$
$\newcommand{\udt}{^{+2.8}_{-1.0}}$
$\newcommand{\abUL}{0.41}$
$\newcommand{\fLL}{0.40}$
$\newcommand{\fabLL}{0.37}$
$\newcommand{\ecoswnosign}{0.0025}$
$\newcommand{\ecosw}{+0.0025}$
$\newcommand{\uecosw}{^{+0.0049}_{-0.0018}}$
$\newcommand{\psurfUL}{8.9}$
$\newcommand{\age}{3.9}$
$\newcommand{\uage}{0.4}$
$\newcommand{\referee}$</div>



<div id="title">

# OrCAS: Origins, Compositions, and Atmospheres of Sub-neptunes. I.  Survey Definition

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2411.16836-b31b1b.svg)](https://arxiv.org/abs/2411.16836)<mark>Appeared on: 2024-11-27</mark> -  _20 pages, 4 figures, 4 tables, 26 sub-Neptunes, 31 TOIs. Accepted to AJ_

</div>
<div id="authors">

<mark>I. J. M. Crossfield</mark>, et al. -- incl., <mark>L. Kreidberg</mark>

</div>
<div id="abstract">

**Abstract:** Sub-Neptunes --- volatile-rich exoplanets smaller than Neptune --- areintrinsically the most common type of planet known.  However, theformation and nature of these objects, as well as the distinctionsbetween sub-classes (if any), remain unclear.  Two powerful tools totease out the secrets of these worlds are measurements of (i)atmospheric composition and structure revealed by transit and/oreclipse spectroscopy, and (ii) mass, radius, and density revealed bytransit photometry and Doppler spectroscopy.  Here we present OrCAS, asurvey to better elucidate the origins, compositions, and atmospheresof sub-Neptunes.  This radial velocity survey uses a repeatable,quantifiable metric to select targets suitable for subsequenttransmission spectroscopy and address key science themes about theatmospheric \& internal compositions and architectures of thesesystems.  Our survey targets $\ntarg$ systems with transitingsub-Neptune planet candidates, with the overarching goal of increasingthe sample of such planets suitable for subsequent atmosphericcharacterization.  This paper lays out our survey's science goals,defines our target prioritization metric, and performs light-curvefits and statistical validation using existing $\TESS$ photometry andground-based follow-up observations.  Our survey serves to continueexpanding the sample of small exoplanets with well-measured propertiesorbiting nearby bright stars, ensuring fruitful studies of thesesystems for many years to come.

</div>

<div id="div_fig1">

<img src="tmp_2411.16836/./triptych_ksms_26targs.png" alt="Fig3" width="100%"/>

**Figure 3. -**  Target prioritization. * Left:* Planet occurrence rate
   ([Fulton and Petigura 2018]()) , showing that sup-Neptunes and super-Earths are
  the most common planets on short-period orbits. * Center:* Gold
  hexagons show transiting exoplanets targeted by JWST in Cycles
  1 \& 2 (our survey began before Cycle 3 results were announced); the shading is a Kernel Density Estimate map showing that hot
  Jupiters and super-Earths were the most commonly targeted
  planets. * Right:* Difference of the two heat maps, showing that
  temperate sub-Neptunes (red region) remain underrepresented by JWST
  relative to the intrinsic underlying population. We identified our
  targets (gray points) by weighting by this under-representation, TSM,
  and observing efficiency (see text for details).
 (*fig:triptych*)

</div>
<div id="div_fig2">

<img src="tmp_2411.16836/./triptych_ksms_26targs.png" alt="Fig6" width="100%"/>

**Figure 6. -**  Target prioritization. * Left:* Planet occurrence rate
   ([Fulton and Petigura 2018]()) , showing that sup-Neptunes and super-Earths are
  the most common planets on short-period orbits. * Center:* Gold
  hexagons show transiting exoplanets targeted by JWST in Cycles
  1 \& 2 (our survey began before Cycle 3 results were announced); the shading is a Kernel Density Estimate map showing that hot
  Jupiters and super-Earths were the most commonly targeted
  planets. * Right:* Difference of the two heat maps, showing that
  temperate sub-Neptunes (red region) remain underrepresented by JWST
  relative to the intrinsic underlying population. We identified our
  targets (gray points) by weighting by this under-representation, TSM,
  and observing efficiency (see text for details).
 (*fig:triptych*)

</div>
<div id="div_fig3">

<img src="tmp_2411.16836/./sample_distribution_fits_v2.png" alt="Fig5" width="100%"/>

**Figure 5. -** Our target sample, showing the distribution of fit planet
  size, temperature, and stellar $T_\mathrm{eff}$. Red points are our
  primary sample, white points are known companions to our targets,
  dashed lines connect  planets orbiting the same target star, and light gray
  points are other known exoplanets. The histogram at * right*
  shows how our planet sizes (red) compare to the intrinsic occurrence
  of sub-Neptunes \citep[gray,][]{fulton:2018}. Noted that these
  best-fit radii differ from the TESS-reported radii plotted in
  Fig. \ref{fig:triptych}c.
 (*fig:sample*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2411.16836"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\svgwidth}{.9\linewidth}$</div>



<div id="title">

# Water-rich sub-Neptunes and rocky super Earths around different Stars: Radii shaped by Volatile Partitioning, Formation, and Evolution

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2411.16879-b31b1b.svg)](https://arxiv.org/abs/2411.16879)<mark>Appeared on: 2024-11-27</mark> -  _25 pages, 17 figures, submitted to A&A_

</div>
<div id="authors">

<mark>R. Burn</mark>, et al.

</div>
<div id="abstract">

**Abstract:** Despite precise characterization measurements, the nature of planets with radii ranging from 2 to 4 Earth radii, the sub-Neptunes, remains unknown due to degeneracies in interior models. However, a statistical ensemble of small planets with measured masses and radii around different stars has been compiled by the field. It can be used to test the prediction of large water reservoirs on sub-Neptunes by planet formation theory with orbital migration. We want to find out whether this water reservoir is included in photoevaporative winds and how much of it can partition into the rocky and metallic interior. We couple the result of a planetary formation model with planetesimal and gas accretion as well as orbital migration to evolution models which assume perfect mixing of water with H/He in the envelope or complete segregation. For the mixed envelopes, we also include an analytic treatment of fractionation during photoevaporative mass-loss. Further, the effect of equilibrium dissolution of water into an assumed magma ocean and into the metallic core is studied for the first time in coupled formation-evolution models. Out of the tested scenarios, the mass-radius relation of exoplanets is best matched under the mixed assumption without water sequestration to the interior. We quantify the radius valley location and scaling with stellar mass. Fractionation is not found to significantly alter the composition of the planets for our initial conditions due to initially massive envelopes on all planets. In contrast, water sequestration has a profound effect on the radius evolution and compositional budget of the planets. The model predicts the preservation of large quantities of water even if the gaseous envelope is lost. Planets with corresponding bulk densities are not observed in comparably large numbers. By combining formation and evolution model, we probe a parameter space favored by core accretion theory. We conclude that observationally searching for the imprint of fractionation on present-day exoplanet atmospheres can constrain initial gas mass fractions of exoplanets but not the occurrence of photoevaporative mass loss in general. The dissolution of different volatiles into the planetary interior and solidification of the magma ocean are natural next steps for comprehensive treatment of atmosphere-interior interaction.

</div>

<div id="div_fig1">

<img src="tmp_2411.16879/./M_R_four_models.png" alt="Fig13" width="100%"/>

**Figure 13. -** Mass versus radius diagram of observed and synthetic planets. The four panels show the resulting planets using different model variations contrasted against the observational data from Parc2024. The synthetic data for all models is sampled according to the stellar mass distribution of the observations and an estimate of the detection and transit probability is applied. More saturated dots imply that the planet was sampled multiple times due to being more likely to be detected and their color is given by the planets' water mass fraction. (*fig:m_R_all*)

</div>
<div id="div_fig2">

<img src="tmp_2411.16879/./period_radius_0p1__mixed_more_HHe_Tsonic3k_redJohn_lim_0p99.png" alt="Fig4.1" width="20%"/><img src="tmp_2411.16879/./period_radius_0p3__mixed_more_HHe_Tsonic3k_redJohn_lim_0p99.png" alt="Fig4.2" width="20%"/><img src="tmp_2411.16879/./period_radius_0p5__mixed_more_HHe_Tsonic3k_redJohn_lim_0p99.png" alt="Fig4.3" width="20%"/><img src="tmp_2411.16879/./period_radius_0p7__mixed_more_HHe_Tsonic3k_redJohn_lim_0p99.png" alt="Fig4.4" width="20%"/><img src="tmp_2411.16879/./period_radius_1p0__mixed_more_HHe_Tsonic3k_redJohn_lim_0p99.png" alt="Fig4.5" width="20%"/>

**Figure 4. -** Period versus radius of unbiased, synthetic planets using the mixed model around stars of different masses. For each stellar mass (top left), a gap in the data was fitted (\texttt{gapfit}, \citealp{Loyd2020}) and the resulting slope ($u_{\rm gap}$) and locus ($R_{\rm gap}$) in logarithmic space are listed. The gray line marks $a_{\rm runaway}$(Eq. \ref{eq:runaway}) within which runaway heating occurs. (*fig:period_radius_Mstars*)

</div>
<div id="div_fig3">

<img src="tmp_2411.16879/./radius_valley_comparison.png" alt="Fig5" width="100%"/>

**Figure 5. -** Comparison of derived loci $R_{\rm gap}$ and slopes with orbital period $u_{\rm gap}$ for the radius valley. Observational data from Petigura2022, [ and é (2022)](http://doi.org/10.1126/science.abl7164), Ho2024, Bonfanti2024 is shown with diamond shaped markers and errors in x-direction showing the range of stellar masses included. From this work, only the Mixed (light blue squares) and Layered (red squares) model output is included here since the Fractionation model gives similar results to the Mixed case and for the Sequestration model no trustworthy fit can be obtained. For the same reason, Venturini2024(dark blue) did not report $R_{\rm gap}$ for their sample of modeled planets around low stellar mass. The values for $R_{\rm gap}$ by [ and é (2022)](http://doi.org/10.1126/science.abl7164) is read-off from their figure. The results of Cloutier2020 and Petigura2022 are based on occurrences instead of detections. For Petigura2022 the values for $R_{\rm gap}$ were shifted to 1 day orbital period (incl. Gaussian error propagation). For the results from Cloutier2020, we report their values which were obtained from marginalizing over orbital periods. The values are only comparable to the other derivations because their derived $u_{\rm gap}$ is close to zero. For \citet[][green line and region, $M_{\rm star}<0.6 $M$_{\odot}$]{Bonfanti2024} and \citet[][orange, $0.6$M$_{\odot} \apprle M_{\rm star}\apprle 1.2 $M$_{\odot}$]{Ho2024}, we show their fitted continuous function for $R_{\rm gap}(M_\star)$ with shaded regions marking 16 to 84\% confidence intervals. (*fig:rgap_ugap_comparison*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2411.16879"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\ignore}[1]$
$\newcommand\na{{New Astronomy}}$
$\newcommand\nat{{Nature}}$
$\newcommand\icarus{{Icarus}}$
$\newcommand\pasj{{PASJ}}$
$\newcommand\mnras{{MNRAS}}$
$\newcommand\aap{{A\&A}}$
$\newcommand\nat{{\it Nature}}$
$\newcommand\aass{{A\&AS}}$
$\newcommand\apj{{ApJ}}$
$\newcommand\apjl{{ApJL}}$
$\newcommand\araa{{ARA\&A}}$
$\newcommand\aj{{AJ}}$
$\newcommand\aas{{American Astronomical Society Meeting.}}$
$\newcommand\apjs{{ApJS}}$
$\newcommand\apss{{Ap\&SS}}$
$\newcommand\aaps{{A\&ASS}}$
$\newcommand\jrasc{{JRASC}}$
$\newcommand\pasp{{PASP}}$
$\newcommand\nature{{Nature}}$
$\newcommand\sciam{{Scientific American}}$
$\newcommand\astroph{{astro-ph}}$
$\newcommand\ssr{{Space Sci. Rev.}}$
$\newcommand\psj{{PSJ}}$
$\newcommand{\mdot}{\dot{M}}$
$\newcommand{\micron}{\mum }$</div>



<div id="title">

#  Gas dynamics around a Jupiter mass planet: II. Chemical evolution of circumplanetary material 

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2411.17408-b31b1b.svg)](https://arxiv.org/abs/2411.17408)<mark>Appeared on: 2024-11-27</mark> -  _20 pages, 17 figures, accepted for publication in A&A_

</div>
<div id="authors">

A. J. Cridland, E. Lega, <mark>M. Benisty</mark>

</div>
<div id="abstract">

**Abstract:** In an ongoing effort to understand planet formation the link between the chemistry of the protoplanetary disk and the properties of resulting planets have long been a subject of interest. These connections have generally been made between mature planets and young protoplanetary disks through the carbon-to-oxygen (C/O) ratio. In a rare number of systems, young protoplanets have been found within their natal protoplanetary disks. These systems offer a unique opportunity to directly study the delivery of gas from the protoplanetary disk to the planet. In this work we post-process 3D numerical simulations of an embedded Jupiter-massed planet in its protoplanetary disk to explore the chemical evolution of gas as it flows from the disk to the planet. The relevant dust to this chemical evolution is assumed to be small, co-moving grains with a reduced dust-to-gas ratio indicative of the upper atmosphere of a protoplanetary disk. We find that as the gas enters deep into the planet's gravitational well, it warms significantly (up to $\sim 800$ K), releasing all of the volatile content from the ice phase. This change in phase can influence our understanding of the delivery of volatile species to the atmospheres of giant planets. The primary carbon, oxygen, and sulfur carrying ices: $CO_2$ , $H_2$ O, and $H_2$ S are released into the gas phase and along with the warm gas temperatures near the embedded planets lead to the production of unique species like CS, SO, and $SO_2$ compared to the protoplanetary disk. We compute the column densities of SO, $SO_2$ , CS, and $H_2$ CS  in our model and find that their values  are consistent with previous observational studies.

</div>

<div id="div_fig1">

<img src="tmp_2411.17408/./Sm01_Obearing_moded.png" alt="Fig11" width="100%"/>

**Figure 11. -** Evolution of the most abundant oxygen bearing species for the 3 test streamlines. The solid lines represent the gas phase of each species while the dashed lines represent the ice species. $CH_2$$O_2$ represents the most abundant `organic' species in the CPD. Note the change in y-axis range between the top and bottom rows. For water and $CH_2$$O_2$ the opacity of the line for $t>40$ orbits is slowly reduced to better see the other lines on the plot. (*fig:Obearing*)

</div>
<div id="div_fig2">

<img src="tmp_2411.17408/./Sm01_cstreams_sample_streams.png" alt="Fig8" width="100%"/>

**Figure 8. -** Example of some of the streamlines by the embedded planet. Three types of streamlines are shown: one misses the CPD completely [(1) blue], one enters into the region of the CPD for less than an orbit [(2) orange], and one that ends up in the CPD [(3) red]. The solid lines represent forward integration from the initial conditions (black circles), while the dashed lines represent backward integration from the initial conditions. The coloured triangles show the location from which the chemical calculation is initialized. The host star is the black square. The x-axis corresponds to the radial distance along the axis passing through the host star and the embedded planet, centred here on the embedded planet. The y-axis runs perpendicular to the host star - planet axis and the z-axis extends from the midplane to heights above the disk. $x > 0$ corresponds to distance farther from the host star than the planet, while $y>0$ corresponds to the leading side of the planet's orbit. (*fig:cstreams*)

</div>
<div id="div_fig3">

<img src="tmp_2411.17408/./Sm01_tempevo_combined.png" alt="Fig3" width="100%"/>

**Figure 3. -** Temperature evolution for test streamlines. The first column shows the temperature as a function of time in orbits of the embedded planet. The second column shows the same temperature evolution of the streamlines as function of distance from the planet. The streamlines that enter the CPD first do so at around the same time ($\sim40$ orbits) because their forward integration is started just before they enter the CPD. In the second row the opacity of the line for $t>40$ orbits is reduced as a function of time to better differentiate separate orbits in the CPD. (*fig:temp*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2411.17408"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

257  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
